# AirBnB Seasonal Trend Analysis

Sean Bowen, Bates Crowther, Jeremy Green, Charlie Rudy, and Aidan Price

In [ ]:
%%configure -f
{"executorMemory": "2G","driverMemory":"1G","executorCores": 2,"numExecutors": 20, 
 "conf": {"spark.dynamicAllocation.enabled": "false", 
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type": "native",
          "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv"}}

# Creating AirBnB Dataset

Cities used in our analysis: Boston, Washington DC, San Diego, Denver, New Orleans, and Nashville

For each city we we used the following datasets: Calendar.csv and listings.csv

Steps taken:

1) Joined the calendar and listings files together for each city

2) Unionized all data to create a dataframe called airbnb

In [ ]:
from pyspark.sql.functions import *

In [ ]:
%%pretty
denverListing = spark.read.format("csv").load('s3://msbx5420-spr23/team9/listings_Denver.csv', header = True)
denverListing = denverListing.withColumn("location", lit("Denver"))
denverListing = denverListing.withColumnRenamed('id', 'listing_id')
# denverListing.show()

In [ ]:
%%pretty
NewOrleansListing = spark.read.format("csv").load('s3://msbx5420-spr23/team9/listings_NewOrleans.csv', header = True)
NewOrleansListing = NewOrleansListing.withColumn("location", lit("New Orleans"))
NewOrleansListing = NewOrleansListing.withColumnRenamed('id', 'listing_id')
#NewOrleansListing.show()

In [ ]:
%%pretty
NashvilleListing = spark.read.format("csv").load('s3://msbx5420-spr23/team9/listings_Nashville.csv', header = True)
NashvilleListing = NashvilleListing.withColumn("location", lit("Nashville"))
NashvilleListing = NashvilleListing.withColumnRenamed('id', 'listing_id')
# NashvilleListing.show()

In [ ]:
%%pretty
DCListing = spark.read.format("csv").load('s3://msbx5420-spr23/team9/listings_DC.csv', header = True)
DCListing = DCListing.withColumn("location", lit("DC"))
DCListing = DCListing.withColumnRenamed('id', 'listing_id')
#DCListing.show()

In [ ]:
%%pretty
SanDiegoListing = spark.read.format("csv").load('s3://msbx5420-spr23/team9/listings_SanDiego.csv', header = True)
SanDiegoListing = SanDiegoListing.withColumn("location", lit("San Diego"))
SanDiegoListing = SanDiegoListing.withColumnRenamed('id', 'listing_id')
#SanDiegoListing.show()

In [ ]:
%%pretty
BostonListing = spark.read.format("csv").load('s3://msbx5420-spr23/team9/listings_boston.csv', header = True)
BostonListing = BostonListing.withColumn("location", lit("Boston"))
BostonListing = BostonListing.withColumnRenamed('id', 'listing_id')
#BostonListing.show()

Loading in the calendars 

In [ ]:
%%pretty
DenverCalendar = spark.read.format("csv").load('s3://msbx5420-spr23/team9/calendar_Denver.csv', header = True)
DenverCalendar = DenverCalendar.dropDuplicates(subset=['listing_id'])
#DenverCalendar.show()

In [ ]:
%%pretty
joined_denver = DenverCalendar.join(denverListing, on='listing_id', how='inner')
#joined_denver.show()

In [ ]:
%%pretty
NewOrleansCalendar = spark.read.format("csv").load('s3://msbx5420-spr23/team9/calendar_NewOrleans.csv', header = True)
NewOrleansCalendar = NewOrleansCalendar.dropDuplicates(subset=['listing_id'])
# NewOrleansCalendar.show()

In [ ]:
%%pretty
joined_newOrleans = NewOrleansCalendar.join(NewOrleansListing, on='listing_id', how='inner')
# joined_newOrleans.show()

In [ ]:
%%pretty
NashvilleCalendar = spark.read.format("csv").load('s3://msbx5420-spr23/team9/calendar_Nashville.csv', header = True)
NashvilleCalendar = NashvilleCalendar.dropDuplicates(subset=['listing_id'])
#NashvilleCalendar.show()

In [ ]:
%%pretty
joined_Nashville = NashvilleCalendar.join(NashvilleListing, on='listing_id', how='inner')
#joined_Nashville.show()

In [ ]:
%%pretty
DCCalendar = spark.read.format("csv").load('s3://msbx5420-spr23/team9/calendar_DC.csv', header = True)
DCCalendar = DCCalendar.dropDuplicates(subset=['listing_id'])
#DCCalendar.show()

In [ ]:
%%pretty
joined_DC = DCCalendar.join(DCListing, on='listing_id', how='inner')
#joined_DC.show()

In [ ]:
%%pretty
SanDiegoCalendar = spark.read.format("csv").load('s3://msbx5420-spr23/team9/calendar_SanDiego.csv', header = True)
SanDiegoCalendar = SanDiegoCalendar.dropDuplicates(subset=['listing_id'])
#SanDiegoCalendar.show()

In [ ]:
%%pretty
joined_SanDiego = SanDiegoCalendar.join(SanDiegoListing, on='listing_id', how='inner')
#joined_SanDiego.show()

In [ ]:
%%pretty
BostonCalendar = spark.read.format("csv").load('s3://msbx5420-spr23/team9/calendar_Boston.csv', header = True)
BostonCalendar = BostonCalendar.dropDuplicates(subset=['listing_id'])
#BostonCalendar.show()

In [ ]:
%%pretty
joined_boston = BostonCalendar.join(BostonListing, on='listing_id', how='inner')
#joined_boston.show()

In [ ]:
%%pretty

airbnb = joined_denver.union(joined_newOrleans).union(joined_Nashville).union(joined_DC).union(joined_SanDiego).union(joined_boston)

airbnb.show()

# Data Cleaning

Only included:

- Entire Homes and Apartments (Excluded private rooms)

- AirBnB's available for 3 months or more (Excluded any that we only available for one season)

Added the following columns: Year, Month, Week, and Season

In [ ]:
%%pretty

clean = airbnb.filter(col('room_type') == "Entire home/apt")
clean = clean.filter(col('availability_365') > 72)

clean = clean.withColumn("adjusted_price", regexp_replace("adjusted_price", "\\$", "").cast("float"))

# Create year and month columns
year_month = airbnb.select(year(airbnb.date).alias('year'), month(airbnb.date).alias('month'),\
                           weekofyear(airbnb.date).alias("week"))
clean = clean.join(year_month)

# Create a season column
seasons = {
        'Winter': [12, 1, 2],
        'Spring': [3, 4, 5],
        'Summer': [6, 7, 8],
        'Fall': [9, 10, 11]
        }

# Create a new column with the season based on the month
clean = clean.withColumn('season', 
                         when(col('month').isin(seasons['Winter']), 'Winter')
                         .when(col('month').isin(seasons['Spring']), 'Spring')
                         .when(col('month').isin(seasons['Summer']), 'Summer')
                         .otherwise('Fall'))

clean.show()

In [ ]:
%%pretty 
# By City
avg_by_city = clean.groupBy('location').agg(mean("adjusted_price").alias("avg_price"),\
                                                              mean('number_of_reviews').alias('avg_reviews'),\
                                                              mean('availability_365').alias('avg_availability'))\
                                                              .orderBy('location')


avg_occupancy_rate = clean.groupBy("location").agg((1 - (mean('availability_365') / 365))\
                                                         .alias("avg_occupancy_rate"))

avg_by_city = avg_by_city.join(avg_occupancy_rate, on="location")
avg_by_city.show(truncate=False)

In [ ]:
%%pretty

# By Location & Season
avg_by_loc_season = clean.groupBy(["location", "season"]).agg(mean("adjusted_price").alias("avg_price"),\
                                                              mean('number_of_reviews').alias('avg_reviews'),\
                                                              mean('availability_365').alias('avg_availability'))\
                                                              .orderBy('location','season')


avg_occupancy_rate = clean.groupBy(["location","season"]).agg((1 - (mean('availability_365') / 365))\
                                                         .alias("avg_occupancy_rate"))

avg_by_loc_season = avg_by_loc_season.join(avg_occupancy_rate, on=["location","season"])
avg_by_loc_season.show(truncate=False)

In [ ]:
%%pretty

#By Location & Week
avg_by_loc_week = clean.groupBy(["neighbourhood", "week"]).agg(mean("adjusted_price").alias("avg_price"),\
                                                              mean('number_of_reviews').alias('avg_reviews'),\
                                                              mean('availability_365').alias('avg_availability'))\
                                                              .orderBy('neighbourhood','week')


avg_occupancy_rate = clean.groupBy(["neighbourhood","week"]).agg((1 - (mean('availability_365') / 365))\
                                                         .alias("avg_occupancy_rate"))

avg_by_loc_week = avg_by_loc_week.join(avg_occupancy_rate, on=["neighbourhood","week"])
avg_by_loc_week.show(truncate=False)

## Filtered By City

In [ ]:
%%pretty
# Overview


avg_by_city = clean.groupBy("location").agg(mean("adjusted_price").alias("avg_price"),\
                                                              mean('number_of_reviews').alias('avg_reviews'),\
                                                              mean('availability_365').alias('avg_availability'))\
                                                              .orderBy('location')
avg_occupancy_rate = clean.groupBy("location").agg((1 - (mean('availability_365') / 365))\
                                                         .alias("avg_occupancy_rate"))

avg_by_city = avg_by_city.join(avg_occupancy_rate, on="location")
avg_by_city.show(truncate=False)

In [ ]:
%%pretty
# New Orleans Seasons

newOrleans = avg_by_loc_season.filter(col("location")=="New Orleans").orderBy('location')
newOrleans.show()

In [ ]:
%%pretty
# Boston Seasons

boston = avg_by_loc_season.filter(col("location")=="Boston").orderBy('location')
boston.show()

In [ ]:
%%pretty
# Denver Seasons

denver = avg_by_loc_season.filter(col("location")=="Denver").orderBy('location')
denver.show()

In [ ]:
%%pretty
# Nashville Seasons

nashville = avg_by_loc_season.filter(col("location")=="Nashville").orderBy('location')
nashville.show()

In [ ]:
%%pretty
# DC Seasons

dc = avg_by_loc_season.filter(col("location")=="DC").orderBy('location')
dc.show()

In [ ]:
%%pretty
# San Diego Seasons

sanDiego = avg_by_loc_season.filter(col("location")=="San Diego").orderBy('location')
sanDiego.show()

## Filtered By Season

In [ ]:
%%pretty
# Overview


avg_by_season = clean.groupBy("season").agg(mean("adjusted_price").alias("avg_price"),\
                                                              mean('number_of_reviews').alias('avg_reviews'),\
                                                              mean('availability_365').alias('avg_availability'))\
                                                              .orderBy('season')
avg_occupancy_rate = clean.groupBy("season").agg((1 - (mean('availability_365') / 365))\
                                                         .alias("avg_occupancy_rate"))

avg_by_season = avg_by_season.join(avg_occupancy_rate, on="season")
avg_by_season.show(truncate=False)

In [ ]:
%%pretty

summer = avg_by_loc_season.filter(col("season") == 'Summer').orderBy("avg_occupancy_rate", ascending = False)
summer.show()

In [ ]:
%%pretty

fall = avg_by_loc_season.filter(col("season") == 'Fall').orderBy("avg_occupancy_rate", ascending = False)
fall.show()

In [ ]:
%%pretty

winter = avg_by_loc_season.filter(col("season") == 'Winter').orderBy("avg_occupancy_rate", ascending = False)
winter.show()

In [ ]:
%%pretty

spring = avg_by_loc_season.filter(col("season") == 'Spring').orderBy("avg_occupancy_rate", ascending = False)
spring.show()

## By Month

In [ ]:
%%pretty

# Overall by Season
avg_by_month_loc = clean.groupBy(['month','location']).agg(mean("adjusted_price").alias("avg_price"),\
                                              mean('number_of_reviews').alias('avg_reviews'),\
                                              mean('availability_365').alias('avg_availability'))\
                                              .orderBy(['month','location'])
avg_occupancy_rate = clean.groupBy(["month",'location']).agg((1 - (mean('availability_365') / 365))\
                                                         .alias("avg_occupancy_rate"))

avg_by_month_loc = avg_by_month_loc.join(avg_occupancy_rate, on=["month",'location'])
avg_by_month_loc.show(truncate=False)

In [ ]:
%%pretty
jan = avg_by_month_loc.filter(col("month")==1).orderBy('avg_occupancy_rate', ascending = False)
jan.show()

In [ ]:
%%pretty
feb = avg_by_month_loc.filter(col("month")==2).orderBy('avg_occupancy_rate', ascending = False)
feb.show()

In [ ]:
%%pretty
mar = avg_by_month_loc.filter(col("month")==3).orderBy('avg_occupancy_rate', ascending = False)
mar.show()

In [ ]:
%%pretty
apr = avg_by_month_loc.filter(col('month')==4).orderBy('avg_occupancy_rate', ascending = False)
apr.show()

In [ ]:
%%pretty
may = avg_by_month_loc.filter(col('month')==5).orderBy('avg_occupancy_rate', ascending = False)
may.show()

In [ ]:
%%pretty
jun = avg_by_month_loc.filter(col('month')==6).orderBy('avg_occupancy_rate', ascending = False)
jun.show()

In [ ]:
%%pretty
jul = avg_by_month_loc.filter(col('month')==7).orderBy('avg_occupancy_rate', ascending = False)
jul.show()

In [ ]:
%%pretty
aug = avg_by_month_loc.filter(col('month')==8).orderBy('avg_occupancy_rate', ascending = False)
aug.show()

In [ ]:
%%pretty
sept = avg_by_month_loc.filter(col('month')==9).orderBy('avg_occupancy_rate', ascending = False)
sept.show()

In [ ]:
%%pretty
octb = avg_by_month_loc.filter(col('month')==10).orderBy('avg_occupancy_rate', ascending = False)
octb.show()

In [ ]:
%%pretty
nov = avg_by_month_loc.filter(col('month')==11).orderBy('avg_occupancy_rate', ascending = False)
nov.show()

In [ ]:
%%pretty
dec = avg_by_month_loc.filter(col('month')==12).orderBy('avg_occupancy_rate', ascending = False)
dec.show()

In [ ]:
%%pretty

# Denver - Most popular city by month
denver_month = avg_by_month_loc.filter(col('location')=='Denver').orderBy('month')
denver_month.show()